In [2]:
import numpy as np

import torch
import torch.nn as nn

from tqdm import tqdm
import copy

import sys, os

import json

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
data_source = 'Pleiade'
base_path = '/Users/alberic/Desktop/divers/projects/hvac_opt/'
data_path = os.path.join(base_path, 'weather_model', 'processed_data', data_source)
with open(os.path.join(data_path, 'normalization.json')) as f:
    normalization_data = json.load(f)

i_start_out = 4
i_end_out = 5

In [4]:
from regressive_model import MLPModel, Weather_dataModule
future_window = 32
past_window = 4
n_features = 19
n_predic = (i_end_out - i_start_out)
dataModule = Weather_dataModule(batch_train_size=128, batch_valid_size=128, past_window=past_window)

model = MLPModel(d_input=n_features*past_window, d_output=n_predic,
                    d_layers=[32, 4], dropout=0.0)
model.load_state_dict(torch.load('/Users/alberic/Desktop/divers/projects/hvac_opt/weather_model/saved_models/Pleiade/17.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [6]:
## Testing loop ##
## Testing loop ##

# Create figure of all predictions
model.dropout = nn.Dropout(0.0)
test_data = dataModule.valid_dataset.data

time = test_data[:,0]*(12*31*24) + test_data[:,1]*(31*24) + test_data[:,2]*(24) + test_data[:,3]
time -= time[0]

def denormalize(output):

    output = copy.copy(output)

    # output[:,0] *= normalization_data['humidity'][1]
    # output[:,0] += normalization_data['humidity'][0]
    
    # output[:,1] *= normalization_data['temperature'][1]
    # output[:,1] += normalization_data['temperature'][0]

    output[:,0] *= normalization_data['temperature'][1]
    output[:,0] += normalization_data['temperature'][0]

    return output

rmse_future = np.zeros(n_predic)

fig = make_subplots(rows=n_predic, cols=1)

test_data_denorm = denormalize(test_data[:, i_start_out:i_end_out])
# fig.add_trace(go.Scatter(x=time, y=test_data_denorm[:,0], name='Humidity'), row=1, col=1)
# fig.add_trace(go.Scatter(x=time, y=test_data_denorm[:,1], name='Temperature'), row=2, col=1)
fig.add_trace(go.Scatter(x=time, y=test_data_denorm[:,0], name='Temperature'), row=1, col=1)


for i in tqdm(range(0, len(test_data)-future_window-(past_window-1))):

    input = copy.copy(test_data[i:i+past_window].flatten())
    outputs = np.zeros((future_window, n_predic))
    for j in range(future_window):

        with torch.no_grad():
            outputs[j] = model(torch.tensor(input).unsqueeze(0).type(torch.float))[0]
            # print(torch.tensor(input).type(torch.float))

            if i+past_window+j<len(test_data):
                input[:-n_features] = input[n_features:]
                input[-n_features:] = copy.copy(test_data[i+j+past_window])
                input[-n_features+i_start_out:-n_features+i_end_out] = outputs[j]

    rmse_future += np.mean(np.sqrt((denormalize(outputs) - test_data_denorm[i+past_window:i+past_window+future_window])**2), axis=0)

    if i%future_window==0:
        outputs = np.concatenate((test_data[i:i+past_window,i_start_out:i_end_out], outputs))
        outputs_denorm = denormalize(outputs)
        color = ['orange', 'green'][int((i/future_window)%2)]
        # fig.add_trace(go.Scatter(x=time[i:i+past_window+future_window], y=outputs_denorm[:, 0], opacity=0.5, marker_color=color, showlegend=False), row=1, col=1)
        # fig.add_trace(go.Scatter(x=time[i:i+past_window+future_window], y=outputs_denorm[:, 1], opacity=0.5, marker_color=color, showlegend=False), row=2, col=1)
        fig.add_trace(go.Scatter(x=time[i:i+past_window+future_window], y=outputs_denorm[:, 0], opacity=0.5, marker_color=color, showlegend=False), row=1, col=1)


rmse_future /= len(test_data)-future_window


# fig.update_layout(title=f'Temperature and humidity over time [hr], rmse over one hour: humidity={rmse_future[0]:.3f} and temperature={rmse_future[1]:.3f}',
#                   height=1000)
fig.update_layout(title=f'Temperature over time [hr], rmse over one hour: temperature={rmse_future[0]:.3f}',
                    height=1000)

fig.show()

100%|██████████| 16380/16380 [00:21<00:00, 771.93it/s]


In [13]:
0.98**100

0.13261955589475294